# D. Langkah Praktikum
Praktikum ini akan memandu Anda membuat aplikasi pencatat pengeluaran harian dari awal hingga akhir, menggunakan struktur modular, kelas OOP, database SQLite, dan antarmuka web Streamlit. 
Penjelasan Kelas Diagram (Diagram Komponen): 
Diagram ini menunjukkan struktur akhir aplikasi yang modular: 
•	streamlit_app.py: Lapisan presentasi (UI) yang menggunakan Streamlit dan memanggil lapisan logika bisnis. 
•	manajer_anggaran.py: Lapisan logika bisnis/repository, menggunakan model dan utilitas database. 
•	model.py: Lapisan data, mendefinisikan struktur Transaksi. 
•	database.py: Lapisan akses data, menangani interaksi SQLite. 
•	konfigurasi.py: Menyimpan konstanta. 
•	pengeluaran_harian.db: Penyimpanan data fisik. 
•	External Libs: Library yang digunakan. 
Panah menunjukkan dependensi antar modul. Ini adalah arsitektur yang umum digunakan untuk memisahkan tanggung jawab dalam sebuah aplikasi. 

## Langkah 1: Persiapan Awal (Konfigurasi & Setup Database) 
Tujuan: Menyiapkan file konfigurasi dasar dan skrip untuk membuat database SQLite beserta tabelnya. 
 


### file konfigurasi.py

In [ ]:
# konfigurasi.py 
import os 
  
BASE_DIR = os.path.dirname(os.path.abspath(__file__)) 
NAMA_DB = 'pengeluaran_harian.db' 
DB_PATH = os.path.join(BASE_DIR, NAMA_DB) 
KATEGORI_PENGELUARAN = ["Makanan", "Transportasi", "Hiburan", "Tagihan", 
"Belanja", "Kesehatan", "Pendidikan", "Lainnya"] 
KATEGORI_DEFAULT = "Lainnya" 